In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [ ]:
import os
import joblib
data_path = '/home/pavel/P/kaggle_data/talking_data'

In [ ]:
app_events = pd.read_csv(os.path.join(data_path, 'app_events.csv'), usecols=('is_installed', 'event_id', 'app_id'),
                         dtype={'is_installed': np.int8, 'event_id': np.int64, 'app_id': np.int64}, chunksize=1000)

In [ ]:
all_apps = joblib.load('all_apps.joblib')

In [ ]:
res = sp.csr_matrix((1488096, 19237))
events = []

In [ ]:
current_event_id = None
item = None

counter = 0
v = None

for chunk in app_events:
    for row in chunk.values:
        event_id, app_id = row[0], row[1]
        if current_event_id is None:
            current_event_id = event_id
            item = dict(zip(all_apps, np.zeros(len(all_apps))))
        if current_event_id == event_id:
            item[app_id] = 1
        else:
            v = pd.DataFrame(item.items(), columns=('app', 'is')).set_index('app').T.values[0]
            res = sp.vstack([res, v])

            current_event_id = event_id
            item = dict(zip(all_apps, np.zeros(len(all_apps))))
            events.append(current_event_id)
        counter += 1

        if len(counter) % 50000 == 0:
            print counter

if item is not None:
    v = pd.DataFrame(item.items(), columns=('app', 'is')).set_index('app').T.values[0]
    res = sp.hstack([res, v])

In [ ]:
joblib.dump(res, 'sparse_app_is_installed')
joblib.dump(events, 'events')